## Waveform Processing -- DSP Optimization Helper

To find an ideal set of DSP parameters, it's helpful to load raw/dsp/hit files and look at a few waveforms.  The `WaveformBrowser` object uses `ProcessingChain` under the hood to process a few waveforms at a time and show you the result.

In [1]:
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm
import boost_histogram as bh
import pickle as pl
from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama.dsp.WaveformBrowser import WaveformBrowser

# do this to use cage_utils functions
import sys
sys.path.insert(1, '../analysis/')
import cage_utils

In [2]:
run = 244

dsp_id = '06'

user = False
hit = True
cal = True
lowE = False
cut = False

etype = 'trapEftp_cal'

# dsp list for uncalibrated data
# dsp_list = ['energy', 'trapEftp', 'trapEmax', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']

# dsp list for calibrated data
# dsp_list = ['energy', 'trapEftp', 'trapEmax', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']
dsp_list = ['trapEftp', 'trapEftp_cal', 'bl', 'bl_sig', 'bl_slope']

df_raw, dg, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list, lowE=lowE)

print(df_raw)

Radius: 10.0; Angle: 90; Rotary: 0.0
user: False; cal: True; hit: True
Using hit files
loading data for /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1966_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1967_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1968_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1969_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1970_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1971_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1972_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1973_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1974_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1975_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1976_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cyc1977_hit.lh5 /global/cfs/cdirs/m2676/data/cage/LH5/hit/cage_run244_cy

In [3]:
# get raw files for wfs
raw_lh5_dir = dg.lh5_dir
raw_list = raw_lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']
print(raw_list)

1964    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1965    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1966    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1967    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1968    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1969    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1970    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1971    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1972    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1973    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1974    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1975    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1976    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1977    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1978    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1979    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1980    /global/cfs/cdirs/m2676/data/cage/LH5/raw/cage...
1981    /globa

Apply data-cleaning cuts if relevant

In [4]:
if cut == True:
    print(f'Making cut')
    df_cut = cage_utils.apply_DC_Cuts(run, df_raw)
else:
    df_cut = df_raw
    
print(df_cut)

            trapEftp  trapEftp_cal           bl    bl_sig  bl_slope
0         897.843506    407.450745  9300.701172  3.972950  0.982828
1        1332.885498    604.832275  9284.148438  3.963908 -0.752437
2         251.882614    114.296570  9297.083008  3.900383 -0.141008
3          72.196930     32.734013  9288.860352  3.970704  0.089649
4         666.181885    302.327179  9291.550781  4.012023 -0.540781
...              ...           ...          ...       ...       ...
3467048  1310.447388    594.652954  9294.141602  3.786648  0.552398
3467049   334.529419    151.809052  9291.635742  4.006526 -0.627742
3467050   677.759766    307.581299  9290.968750  3.881753 -0.136750
3467051   288.650909    130.985489  9294.819336  3.872201  0.254664
3467052   306.598328    139.131607  9301.790039  3.992154 -0.624039

[3467053 rows x 5 columns]


Make baseline cut if didn't apply other cuts

In [5]:
# baseline cut
if cut:
    if run <79:
        bl_cut_lo, bl_cut_hi = 9150,9320
    if run>79 and run <117:
        bl_cut_lo, bl_cut_hi = 8500, 10000
    if run>=117:
        bl_cut_lo, bl_cut_hi = 9700, 9760

    df_cut = df.query(f'bl > {bl_cut_lo} and bl < {bl_cut_hi}').copy()
    
    # baseline cut
    bl = df['bl']
    bl_cut = (bl > bl_cut_lo) & (bl < bl_cut_hi)

    # print(cut)
    print(bl_cut.value_counts())

In [6]:
# energy cut 
energy = df_cut[etype]
# print(df_cut)

if cal == True:
    energy = df_cut['trapEftp_cal']
    energy_cut = (energy > 1450) & (energy < 1470) # 1460 peak
    # energy_cut = (energy > 60) & (energy < 65) # 60 keV peak
    
else:
    # energy = df_hit['trapEmax']
    energy_cut = (energy > 2805) & (energy < 2835) # 1460 peak, uncalibratedm trapEmax
    # energy_cut = (energy > 2000000) & (energy < 2500000) #where alphas seem to be in run 117, raw energy parameter
    # energy_cut = (energy > 200) & (energy < 1000) #where low e weirdness is in DCR, in trapEftp
    
# print(type(energy))
# print(energy_cut)
print(energy_cut.value_counts())

# energy_raw = df['energy']
# energy_cut_raw = (energy_raw > 10200) & (energy_raw < 10600)

# -- set the cut to be used when viewing waveforms -- 
# cut = bl_cut & energy_cut
cut = energy_cut
# print(cut)
print(cut.value_counts())

False    3419780
True       47273
Name: trapEftp_cal, dtype: int64
False    3419780
True       47273
Name: trapEftp_cal, dtype: int64




## DCR / Pole-Zero Correction Plots

Show the raw waveform and the pole-zero corrected waveform used to calculate the DCR (tail slope) parameters.


In [7]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
print(dsp_id)
browser = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          # database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pzDCR', 'dcr_trap', 'dcr_trap'], # names of waveforms from dsp config file
                          selection = cut, # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "DCR: {dcr:0.2f}"],
                          legend_opts={'loc':"lower right"},
                          lines=['dcr'], # add hlines and vlines
                          x_lim=(38000, 80000) # x axis range
                          )
# plt.rcParams['figure.figsize'] = [14, 10]

06


JSONDecodeError: Expecting ',' delimiter: line 10 column 5 (char 390)

In [ ]:
%matplotlib widget
browser.draw_next()
plt.ylim(3100, 3300)
plt.axvline(42500, c='r')
plt.axvline(55000, c='r')

## Energy Plots

In [32]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, 
                           waveforms =['wf_blsub', 'wf_pz', 'wf_trap'],
                           selection = cut, 
                           wf_styles = [{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                           legend = ['Waveform', 'PZ Corrected',  'trapEftp = {trapEftp:0.2f}'],
                           legend_opts = {'loc':"lower left"},
                           lines = ['trapEftp'],
                           x_lim = (0, 80000)
                           )

Processing parameters: ['bl, bl_sig', 'wf_blsub', 'wf_pz', 'wf_trap', 'wf_atrap', 'tp_max', 'tp_0', 'trapEftp']
Required input parameters: ['waveform']
Copied output parameters: []
Processed output parameters: ['wf_blsub', 'wf_pz', 'wf_trap', 'trapEftp']
Database lookup: using default value of 51.00*us for db.pz2.tau1
Database lookup: using default value of 4*us for db.pz2.tau2
Database lookup: using default value of 0.05 for db.pz2.frac


In [47]:
%matplotlib widget
browser2.draw_next()
# plt.ylim(4000, 4500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[705]

## Timepoint calculators -- `tp_0`, etc.

In [34]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser3 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, 
                           waveforms = ['wf_blsub', 'wf_pz', 'wf_atrap'],
                           selection = cut,
                           wf_styles = [{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                           legend = ['Waveform', 'PZ Corrected',  't0 = {tp_0:0.2f}'],
                           lines=['tp_0'],
                           x_lim=(39000, 41000)
                           )

Processing parameters: ['bl, bl_sig', 'wf_blsub', 'wf_pz', 'wf_atrap', 'tp_max', 'tp_0']
Required input parameters: ['waveform']
Copied output parameters: []
Processed output parameters: ['wf_blsub', 'wf_pz', 'wf_atrap', 'tp_0']
Database lookup: using default value of 51.00*us for db.pz2.tau1
Database lookup: using default value of 4*us for db.pz2.tau2
Database lookup: using default value of 0.05 for db.pz2.frac


In [48]:
%matplotlib widget
browser3.draw_next()
# plt.ylim(-10, 200) 
# plt.xlim(38000, 45000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[705]

## T/E calculator

In [49]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser_tri = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                              waveforms = ['wf_blsub', 'wf_pz', 'wf_triangle'],
                              selection = cut,
                              wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                              legend=['Waveform', 'PZ Corrected',  'triE= {triE:0.2f}'],
                              lines=['triE'],
                              x_lim=(0, 50000)
                              )

Processing parameters: ['bl, bl_sig', 'wf_blsub', 'wf_pz', 'wf_triangle', 'triE']
Required input parameters: ['waveform']
Copied output parameters: []
Processed output parameters: ['wf_blsub', 'wf_pz', 'wf_triangle', 'triE']
Database lookup: using default value of 51.00*us for db.pz2.tau1
Database lookup: using default value of 4*us for db.pz2.tau2
Database lookup: using default value of 0.05 for db.pz2.frac


In [50]:
%matplotlib widget
browser_tri.draw_next()
# plt.ylim(-10, 100) 
plt.xlim(38000, 45000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(38000.0, 45000.0)

## WF_psd
(todo)

## LF noise
Low-frequency maximum value `lf_max`

In [51]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser4 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                           waveforms = ['wf_psd', 'wf_psd'],
                           selection = cut,
                           wf_styles = [{'linestyle':['-']}],
                           legend = ['lf_max = {lf_max:0.2f}', 'wf_psd'],
                           lines=['lf_max'],
                           x_unit = 'khz',
                           x_lim=(0, 200),
                           verbosity=2
                           )

Processing parameters: ['bl, bl_sig', 'wf_blsub', 'wf_psd', 'lf_max']
Required input parameters: ['waveform']
Copied output parameters: []
Processed output parameters: ['wf_psd', 'wf_psd', 'lf_max']
Added variable waveform with shape (8, 8192) and type float32
Binding input buffer of shape (128, 8192) and type uint16 to variable waveform with shape (8, 8192) and type float32
Added variable bl with shape (8,) and type float32
Added variable bl_sig with shape (8,) and type float32
Added processor: mean_stdev(waveform[0:2500], bl, bl_sig)
Added variable wf_blsub with shape (8, 8192) and type float32
Added processor: subtract(waveform, bl, wf_blsub)
Added variable wf_psd with shape (8, 1501) and type float32
Building function psd from init_args [array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=f

In [52]:
%matplotlib widget
browser4.draw_next()
# plt.ylim(0, 2000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[625]

## HF PSD

High-frequency maximum value `hf_max`

In [53]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser5 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                           waveforms = ['wf_psd', 'wf_psd'],
                           selection = cut, 
                           wf_styles = [{'linestyle':['-']}],
                           legend=['hf_max = {hf_max:0.2f}', 'wf_psd'],
                           lines=['hf_max'],
                           x_unit = 'mhz',
                           x_lim=(0, 50),
                           verbosity=2
                           )

Processing parameters: ['bl, bl_sig', 'wf_blsub', 'wf_psd', 'hf_max']
Required input parameters: ['waveform']
Copied output parameters: []
Processed output parameters: ['wf_psd', 'wf_psd', 'hf_max']
Added variable waveform with shape (8, 8192) and type float32
Binding input buffer of shape (128, 8192) and type uint16 to variable waveform with shape (8, 8192) and type float32
Added variable bl with shape (8,) and type float32
Added variable bl_sig with shape (8,) and type float32
Added processor: mean_stdev(waveform[0:2500], bl, bl_sig)
Added variable wf_blsub with shape (8, 8192) and type float32
Added processor: subtract(waveform, bl, wf_blsub)
Added variable wf_psd with shape (8, 1501) and type float32
Building function psd from init_args [array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=f

In [54]:
%matplotlib widget
browser5.draw_next()
# plt.ylim(0, 2000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[625]

## Baseline Slope

In [55]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser6 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                           waveforms=['wf_blsub', 'wf_pzDCR', 'bl_trap'],
                           selection = cut,
                           wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                           legend=['Waveform', 'PZ Corrected', "bl_slope: {bl_slope:0.2f}"],
                           lines=['bl_slope'],
                           x_lim=(0, 20000),
                           )

Processing parameters: ['bl, bl_sig', 'wf_blsub', 'wf_pzDCR', 'bl_trap', 'bl_slope']
Required input parameters: ['waveform']
Copied output parameters: []
Processed output parameters: ['wf_blsub', 'wf_pzDCR', 'bl_trap', 'bl_slope']
Database lookup: using default value of 52.00*us for db.pz2.tau1
Database lookup: using default value of 4*us for db.pz2.tau2
Database lookup: using default value of 0.05 for db.pz2.frac


In [56]:
%matplotlib widget
browser6.draw_next()
plt.ylim(-30, 30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-30.0, 30.0)

In [57]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser7 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                           waveforms = ['log_tail'],
                           selection = cut,
                           wf_styles=[{'linestyle':['-']}],
                           legend=['log tail'],
                           # x_lim=(0, 80000)
                           )

Processing parameters: ['bl, bl_sig', 'wf_blsub', 'log_tail']
Required input parameters: ['waveform']
Copied output parameters: []
Processed output parameters: ['log_tail']
Found multiple compatible type signatures for this function: ['ff', 'ff'] Using signature ff.


In [58]:
%matplotlib widget
browser7.draw_next()
# plt.xlim(0, 40000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[625]